In [36]:
import websocket
import pandas as pd
import requests
from pandas.io.json import json_normalize
import json
from code_ref import get_keys
from code_ref import findata
from code_ref import FinnhubRequestException
from code_ref import FinnhubAPIException
import time
import threading
%matplotlib inline
%load_ext autoreload
%autoreload 2
from ratelimit import limits
from ratelimiter import RateLimiter
import numpy as np


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
keys = get_keys("/Users/augustingoudet/.secret/code_file") #Finnhub Key
api_key1 = keys ['finnhub'] #api_key
company = "KER.PA" #Company Ticker
query = findata(api_key = api_key1)
rate_limiter = RateLimiter(max_calls=45, period=60)

In [38]:
url = "https://finnhub.io/api/v1/"
ext1 = "?symbol={}&token={}"
ext2 = "?symbol={}&metric=management&token={}"
ext3 = "?symbol={}&metric=margin&token={}"
ext4 = "?symbol={}&metric=growth&token={}"
ext5 = "?symbol={}&metric=price&token={}"
ext6 = "?symbol={}&metric=valuation&token={}"
ext7 = "?symbol={}&metric=financialStrength&token={}"
ext8 = "?symbol={}&metric=perShare&token={}"
profile ="stock/profile"
ceo_comp = "stock/ceo-compensation"
board = "stock/executive"
reco = "stock/recommendation"
price_target = "stock/price-target"
up_down = "stock/upgrade-downgrade"
peers = "stock/peers"
rev_est = "stock/revenue-estimate"
eps_est = "stock/eps-estimate"
ownership = "stock/investor-ownership"
metric= "stock/metric"
quote = "quote"
news = "news"
covid = "covid19"
transcript = "stock/transcripts"
divs = "stock/dividend?symbol={}&from=2019-02-01&to=2020-02-01"
split = "stock/split?symbol=AAPL&from=2010-02-01&to=2020-02-01"
inc = "stock/financials?symbol={}&statement=is&freq=annual"
bs = "stock/financials?symbol={}&statement=bs&freq=annual"
cfs = "stock/financials?symbol={}&statement=cf&freq=annual"


In [39]:
tickers = pd.read_csv('Ticker_list_1.csv')
tickers.drop(tickers[tickers['Exchange'] == 'PNK'].index, inplace = True)
drop = ['Argentina', 'Brazil', 'Estonia', 'Greece', 'Iceland', 'India', 'Indonesia', 'Latvia', 'Lithuania', 'Malaysia', 'Mexico', 'New Zealand', 'Qatar', 'Russia', 'Taiwan', 'Thailand', 'Turkey','China']
drop1 = ['MUN', 'STU', 'DUS', 'HAM', 'HAN', 'EUX', 'FRA', "ENX", 'BER', 'CNQ', 'VAN', 'IOB', 'KOE', 'PCX', 'OBB']
tickers.dropna(subset = ['Country'], inplace = True)

In [42]:
for values in drop:
    tickers.drop(tickers[tickers['Country'] == values].index, inplace = True)
for exch in drop1: 
    tickers.drop(tickers[tickers['Exchange'] == exch].index, inplace = True)

In [43]:
tickers['Category Name'].fillna(1, inplace = True)
drop2 = tickers[(tickers['Country'] == 'USA') & (tickers['Category Name'] == 1)].index
tickers.drop(drop2, inplace = True)

In [44]:
tickers.drop_duplicates(subset = 'Name', keep = 'first', inplace = True)
tickers.shape

(18025, 5)

In [45]:
tick = list(tickers.Ticker)
tick1 = tick[0:5]

## Metrics 

In [46]:
results = []

for items in tick1:
    with rate_limiter:
        data = url + metric + ext2.format(items, api_key1)
        req = requests.get(data).json()
        with open ('backup/mgmt.py', 'a+') as f:
            json.dump(req,f)
        results.append(req)
        m1 = json_normalize(results)
m1.to_csv('data_api/management1.csv')

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [47]:
results = []

for items in tick1: 
    with rate_limiter:
        data = url + metric + ext3.format(items, api_key1)
        req = requests.get(data).json()
        with open ('backup/margin.py', 'a+') as f:
            json.dump(req,f)
        results.append(req)
        m2 = json_normalize(results)
m2.to_csv('data_api/margin1.csv')

In [48]:
results = []

for items in tick1: 
    with rate_limiter:
        data = url + metric + ext4.format(items, api_key1)
        req = requests.get(data).json()
        with open ('backup/growth.py', 'a+') as f:
            json.dump(req,f)
        results.append(req)
        m3 = json_normalize(results)
m3.to_csv('data_api/growth1.csv')

## Review Later

In [26]:
# results = []

# for items in tick1: 
#     with rate_limiter:
#         data = url + metric + ext6.format(items, api_key1)
#         req = requests.get(data).json()
#         results.append(req)
#         m5 = json_normalize(results)
#         m5.dropna(inplace=True)
# m5

In [24]:
# results = []

# for items in tick1: 
#     with rate_limiter:
#         data = url + metric + ext7.format(items, api_key1)
#         req = requests.get(data).json()
#         results.append(req)
#         m6 = json_normalize(results)
#         m6.dropna(inplace=True)
# m6

In [25]:
# results = []

# for items in tick1: 
#     with rate_limiter:
#         data = url + metric + ext8.format(items, api_key1)
#         req = requests.get(data).json()
#         results.append(req)
#         m7 = json_normalize(results)
#         m7.dropna(inplace=True)
    
# m7

In [16]:
results = []


for items in tick1:
#     @limits(calls = 12, period = ONE_MINUTE)
    data = query.ceo_compensation(symbol = items)
    results.append(data)
    df = pd.DataFrame(results)
    df.dropna(inplace = True)
    time.sleep(65)

%time

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


## Stock Upgrade / Downgrade

In [6]:
url4 = "https://finnhub.io/api/v1/stock/upgrade-downgrade?symbol={}&token={}".format(company, api_key1) #url
req4 = requests.get(url4) 
status = pd.DataFrame(req4.json())

In [7]:
status

,symbol,gradeTime,company,fromGrade,toGrade,action
0,KER.PA,1541116800,RBC Capital,Sector Perform,Outperform,up


### Peers

In [ ]:
url5 = "https://finnhub.io/api/v1/stock/peers?symbol={}&token={}".format(company, api_key) #url
req5 = requests.get(url5) 

In [ ]:
print(req5.json())

## Revenue Estimates

In [ ]:
url6 = "https://finnhub.io/api/v1/stock/revenue-estimate?symbol={}&token={}".format(company, api_key) #url
req6 = requests.get(url6) 
rev = pd.DataFrame(req6.json())
rev1 = json_normalize(rev['data'])
rev1

## Earnings Estimate

In [ ]:
url7 = "https://finnhub.io/api/v1/stock/eps-estimate?symbol={}&token={}".format(company, api_key) #url
req7 = requests.get(url7) 
print(req7.json())

In [ ]:
eps = pd.DataFrame(req7.json())
eps1 = json_normalize(eps['data'])
eps1

## Investor Ownership

In [ ]:
url8 = "https://finnhub.io/api/v1/stock/investor-ownership?symbol={}&token={}".format(company, api_key) #url
req8 = requests.get(url8) 
# print(req8.json())
own = req8.json()

In [ ]:
ownership = pd.DataFrame(own.items(), columns =['Features', 'Value'])
# print(req8.json())
owner = pd.DataFrame(req8.json())
owner1 = json_normalize(owner['ownership'])

In [ ]:
owner1

## Fund Ownership

In [ ]:
url9 = "https://finnhub.io/api/v1/stock/fund-ownership?symbol={}&token={}".format(company, api_key) #url
req9 = requests.get(url9)
fdowner = pd.DataFrame(req9.json())
fdowner1 = json_normalize(fdowner['ownership'])
fdowner1

## stock quote

In [ ]:
url10 = "https://finnhub.io/api/v1/quote?symbol={}&token={}".format(company, api_key) #url
req10 = requests.get(url10) 
print(req10.json())

In [ ]:
url10 = "https://finnhub.io/api/v1/quote?symbol=BINANCE:BTCEUR&token={}".format(api_key) #url
req10 = requests.get(url10) 
print(req10.json())

## General News

In [ ]:
url11 = "https://finnhub.io/api/v1/news?category=general&token={}".format(api_key) #url
req11 = requests.get(url11) 
news = pd.DataFrame(req11.json())
news

## company news 

In [ ]:
url12 = "https://finnhub.io/api/v1/news/{}?token={}".format(company,api_key) #url
req12 = requests.get(url12) 
compnews = pd.DataFrame(req12.json())
compnews

## Major Developments

In [ ]:
url13 = "https://finnhub.io/api/v1/major-development?symbol={}&token={}'".format(company,api_key) #url
req13 = requests.get(url13) 
# print(req13.json())

## New Sentiment

In [ ]:
url14 = "https://finnhub.io/api/v1/news-sentiment?symbol={}&token={}".format(company,api_key) #url
req14 = requests.get(url14) 

cpn = pd.DataFrame(req14.json())
print(req14.json())

## Economic Code & Data


In [ ]:
url15 = "https://finnhub.io/api/v1/economic/code?token={}".format(api_key) #url
req15 = requests.get(url15) 
# print(req15.json())


In [ ]:
code = pd.DataFrame(req15.json())
code.head(n=15)

In [ ]:
df = code.loc[code['country'] == 'United States']

In [ ]:
code.code.value_counts().sum()

In [ ]:
url16 = "https://finnhub.io/api/v1/economic?code=A-FRA-6682&token={}".format(api_key) #url
req16 = requests.get(url16) 
eco = pd.DataFrame(req16.json(), columns =['Date', 'Monthly Bankruptices in France'])
eco

In [ ]:
url17 = "https://finnhub.io/api/v1/covid19/us?token={}".format(api_key) #url
req17 = requests.get(url17) 
# print(req17.json())

covid = pd.DataFrame(req17.json())
covid.head(n=20)

## Financials

In [111]:

import csv, json, sys
if sys.argv[1] is not None and sys.argv[2] is not None:
    fileInput = sys.argv[1]
    fileOutput = sys.argv[2]

In [122]:
import requests
r = requests.get('https://finnhub.io/api/v1/stock/financials?symbol=AAPL&statement=bs&freq=annual&token=bq06smvrh5rddd65esl0')
data = r.json()
print(data)

{'financials': [{'accountsPayable': 46236, 'accountsReceivableTradeNet': 22926, 'accruedExpenses': 20951, 'accumulatedDepreciationTotal': -58579, 'cash': 12204, 'cashEquivalents': 36640, 'cashandShortTermInvestments': 100557, 'commonStockTotal': 45174, 'currentPortofLTDebtCapitalLeases': 10260, 'deferredIncomeTax': 31504, 'goodwillNet': 5414, 'intangiblesNet': 3206, 'longTermDebt': 91807, 'longTermInvestments': 105341, 'notesPayableShortTermDebt': 5980, 'otherCurrentAssetsTotal': 12352, 'otherCurrentliabilitiesTotal': 43242, 'otherEquityTotal': -1291, 'otherLiabilitiesTotal': 50503, 'otherLongTermAssetsTotal': 32978, 'period': '2019-09-28', 'propertyPlantEquipmentTotalGross': 95957, 'propertyPlantEquipmentTotalNet': 37378, 'retainedEarningsAccumulatedDeficit': 45898, 'shortTermInvestments': 51713, 'tangibleBookValueperShare': 20.36534, 'totalAssets': 338516, 'totalCommonSharesOutstanding': 4443.236, 'totalCurrentAssets': 162819, 'totalCurrentLiabilities': 105718, 'totalDebt': 108047, '

In [121]:
key_list = []

for key in data: 
    key_list.append(key)

['financials', 'symbol']

In [123]:
f = csv.writer(open("test.csv", "w"))
f.writerows(key_list)



In [131]:
# def jsontocsv(input_json, output_path):
# keylist = []
# for key in jsonobj[0]:
#     keylist.append(key)
#     f = csv.writer(open(output_path, "w"))
#     f.writerow(keylist)
 
#   for record in jsonobj:
#     currentrecord = []
#     for key in keylist:
#       currentrecord.append(record[key])
#       f.writerow(currentrecord)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)

In [54]:
df = pd.DataFrame(r.json())

In [56]:
df1 = json_normalize(df['financials'])

In [58]:


df1.columns

Index(['accountsPayable', 'accountsReceivableTradeNet', 'accruedExpenses',
       'accumulatedDepreciationTotal', 'cash', 'cashEquivalents',
       'cashandShortTermInvestments', 'commonStockTotal',
       'currentPortofLTDebtCapitalLeases', 'deferredIncomeTax', 'goodwillNet',
       'intangiblesNet', 'longTermDebt', 'longTermInvestments',
       'notesPayableShortTermDebt', 'otherCurrentAssetsTotal',
       'otherCurrentliabilitiesTotal', 'otherEquityTotal',
       'otherLiabilitiesTotal', 'otherLongTermAssetsTotal', 'period',
       'propertyPlantEquipmentTotalGross', 'propertyPlantEquipmentTotalNet',
       'retainedEarningsAccumulatedDeficit', 'shortTermInvestments',
       'tangibleBookValueperShare', 'totalAssets',
       'totalCommonSharesOutstanding', 'totalCurrentAssets',
       'totalCurrentLiabilities', 'totalDebt', 'totalEquity', 'totalInventory',
       'totalLiabilities', 'totalLiabilitiesShareholdersEquity',
       'totalLongTermDebt', 'totalReceivablesNet', 'unrealizedG